# Composite Distance Functions Using Sklearn Nearest Neighbors
This notebook illustrates how to make composite product norms using `sklearn` that can be used for `NearestNeighbor` objects.

### Import Block
For this notebook, we'll be using `sklearn` and `numpy`.

In [ ]:
import numpy as np
import sklearn
import numpy as np
from sklearn.neighbors import KDTree, DistanceMetric, NearestNeighbors

### Define Norm as Product of Norms
Here, we will use a composite product norm composed of a product of norms over given (contiguous) subsets of states. The norms over these subsets are taken to be L2 norms.

If $\mathbf{x} = [\mathbf{s} \; \mathbf{a}]^T$, where $\mathbf{s}$ and $\mathbf{a}$ are the contiguous subsets of $\mathbf{x} \in \mathbb{R}^d$ that this product norm is decomposed into, then this product norm can be written as:

$$d(\mathbf{x}_1, \mathbf{x}_2) = ||\mathbf{s}_1 - \mathbf{s}_2||_2 \times ||\mathbf{a}_1 - \mathbf{a}_2||_2$$

In [ ]:
def composite_product_norm(x1, x2, **kwargs):
    """Function defining a composite product norm over states and actions.

    This norm is intended to be used with GPR kernels that have a product
    decomposition over states and actions. This norm can therefore be used to
    calculate composite L2 distance between states and actions. It is therefore
    only small when similar actions are taken in similar states.

    Parameters:
        x1 (np.array): Array corresponding to the first input vector. Distance
            is calculated as the distance between x1 and x2.
        x2 (np.array): Array corresponding to the second input vector. Distance
            is calculated as the distance between x2 and x1.

    Returns:
        d (float): A float value corresponding to the composite product norm
            distance between x1 and x2 in standardized space. Since points are
            standardized in each dimension, this is a measure of the product of
            state and action similarities.
    """
    # Get state dimension
    ds = kwargs["ds"]

    # Get states and actions of both inputs
    x1_states, x1_actions = x1[:ds], x1[ds:]
    x2_states, x2_actions = x2[:ds], x2[ds:]

    # Now compute product of L2 norms
    return np.linalg.norm(x1_states - x2_states, ord=2) * \
           np.linalg.norm(x1_actions - x2_actions, ord=2)


### Apply Composite Norm Function to Nearest Neighbors 
With our distance metric defined, we are now ready to apply this metric to a `NearestNeighbor` example.

In [ ]:
# Create dataset
X = np.random.normal(loc=0, scale=1, size=(100, 4))  # Normally-distributed data
ds = 2  # Dimension separating subsets

### Fit NearestNeighbor Object Using Composite Metric

In [ ]:
knn = NearestNeighbors(n_neighbors=10, metric=composite_product_norm, 
                       metric_params={"ds": ds}).fit(X)